In [ ]:
import cv2
import tensorflow as tf
import numpy as np

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


train_data = train_datagen.flow_from_directory(
    '/Users/alaaalhrebi/Downloads/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    '/Users/alaaalhrebi/Downloads/dataset/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
image_size = (224, 224)
batch_size = 32

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=10)
model.save('fire_detection_model.h5')

In [ ]:
plt.figure(figsize=(12, 9))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.suptitle("Detecting fires (Fire or Non Fire)")
        plt.axis("off")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/Users/alaaalhrebi/Documents/fire_detection_model.h5')

In [ ]:
img = image.load_img('/Users/alaaalhrebi/Downloads/test_image2.JPG', target_size=image_size)
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # إضافة بعد للدفعة


In [ ]:
img_pil = Image.open('/Users/alaaalhrebi/Downloads/test_image4.jpg').convert('RGB')

In [ ]:
img = np.array(img_pil)

img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

img_resized = cv2.resize(img, (224, 224))

img_array = np.expand_dims(img_resized, axis=0) / 255.0


In [ ]:
predictions = model.predict(img_array)
score = predictions[0][0]


if score > 0.5:
    print( "🔥 Fire Detected")
    
else:
    print("✅ No Fire")
    
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.axis('off')  
plt.show()

In [ ]:
from IPython.display import clear_output
import time
from playsound import playsound
import threading

In [ ]:
def play_alarm():
    playsound('/Users/alaaalhrebi/Downloads/alarm.mp3')  

In [ ]:
cap = cv2.VideoCapture(0)  # 0 = الكاميرا الافتراضية

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ لم يتم التقاط الصورة")
        break


        img_resized = cv2.resize(frame, (224, 224))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)


    prediction = model.predict(img_input)[0][0]

    if prediction > 0.5:
        print("🔥🔥🔥 FIRE DETECTED! 🔥🔥🔥")
        clear_output(wait=True)
        plt.imshow(frame[..., ::-1])
        plt.axis('off')
        plt.show()

        threading.Thread(target=play_alarm).start()
        break  
    else:
        print("✅ No Fire detected. Checking again in 1 minute...")
        clear_output(wait=True)
        plt.imshow(frame[..., ::-1])
        plt.axis('off')
        plt.show()

    time.sleep(60)

cap.release()